Inference comparison for `TinyBert fine tuned model` with and without `Flash Attention`
===========================================

In [1]:
# First install the dependencies
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install xformers flash-attn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 29.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver d

In [2]:
# Load GLUE dataset (MRPC task)
from datasets import load_dataset  # Import the load_dataset function
import evaluate


task = "mrpc"  # Set the task to "mrpc"
dataset = load_dataset("glue", task)
metric = evaluate.load("glue", task)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
# run this cell if you saved model files as zip
!unzip fine-tuned-model.zip -d fine-tuned-model

Archive:  fine-tuned-model.zip
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/special_tokens_map.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/README.md  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/scheduler.pt  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/adapter_model.safetensors  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/vocab.txt  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/adapter_config.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/tokenizer_config.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/rng_state.pth  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/tokenizer.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/training_args.bin  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/trainer_state.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/optimizer.pt  


In [5]:
# Load the trained model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import time

model = AutoModelForSequenceClassification.from_pretrained("fine-tuned-model/model-files")

# Load tokenizer and move the model to CUDA
tokenizer = AutoTokenizer.from_pretrained("fine-tuned-model/model-files")  # Path to your saved model
model.config.use_cache = True
model.eval()  # Set to evaluation mode

config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): lora.Linear(
                (base_layer): Linear(in_features=312, out_features=312, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=312, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_featur

In [6]:
# we will print out the size of the trained model
import torch
from transformers import AutoModel

def calculate_model_size(model):
    """
    Calculate the size of the model in MB.

    :param model: PyTorch model
    :return: Model size in MB
    """
    total_params = sum(param.numel() for param in model.parameters())
    total_size = total_params * 0.5 / (1024 ** 2)  # Each parameter is 4 bytes (float32)
    return total_size

# Calculate and print the model size
model_size = calculate_model_size(model)
print(f"The size of the model is approximately {model_size:.2f} MB.")
model.to('cuda')

The size of the model is approximately 6.90 MB.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): lora.Linear(
                (base_layer): Linear(in_features=312, out_features=312, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=312, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_featur

In [7]:
# write out the helper function to prepare our batches for inputs
import torch
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from flash_attn import flash_attn_func
import torch.nn.functional as F
import numpy as np

from datasets import Dataset


def prepare_batch_inputs(texts1, texts2):
    """
    Prepare batch of input tokens and attention masks

    :param texts: List of input texts to process
    :return: Tokenized batch inputs
    """
    inputs = tokenizer(
        texts1, texts2,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )

    # Move inputs to the same device as the model
    return {k: v.to(device) for k, v in inputs.items()}

In [8]:
# this function applies flash attention to our
# trained model
import torch
from flash_attn import flash_attn_func

def apply_flash_attention(hidden_states, model):
    """
    Apply Flash Attention to 4-bit quantized model's hidden states.

    :param hidden_states: Input hidden states
    :param model: 4-bit quantized model
    :return: Attention output
    """
    batch_size, seq_len, hidden_size = hidden_states.shape
    num_heads = model.config.num_attention_heads
    head_dim = hidden_size // num_heads

    # Ensure contiguous memory and correct device
    hidden_states = hidden_states.contiguous().to(model.device)

    # Reshape for Flash Attention
    hidden_states = hidden_states.view(batch_size, seq_len, num_heads, head_dim)

    # Cast to bfloat16 for Flash Attention compatibility
    hidden_states = hidden_states.to(torch.bfloat16)

    # Apply Flash Attention
    q = k = v = hidden_states
    try:
        attn_output = flash_attn_func(
            q, k, v,
            dropout_p=0.0,
            softmax_scale=None,
            causal=False
        )
    except RuntimeError as e:
        print(f"Flash Attention failed: {e}")
        # Fallback to model's native attention
        return model(inputs_embeds=hidden_states).last_hidden_state

    # Reshape back to original dimensions
    attn_output = attn_output.reshape(batch_size, seq_len, hidden_size)

    return attn_output

In [9]:
# the following class computes inference latency
# with and without flash attention
import torch
from torch.utils.data import Dataset, DataLoader
import time
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer

class MRPCDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        """
        Custom Dataset to wrap the MRPC dataset for DataLoader.
        :param dataset: The MRPC dataset (from `load_dataset`)
        :param tokenizer: The tokenizer used for text preprocessing
        """
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sentence1 = self.dataset[idx]['sentence1']
        sentence2 = self.dataset[idx]['sentence2']
        inputs = self.tokenizer(sentence1, sentence2, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        # Flatten the tensors to remove extra dimensions
        return {key: val.squeeze(0) for key, val in inputs.items()}


def batch_inference_with_flash(dataloader, model, device):
    inference_times = []
    all_predictions = []

    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        for batch in dataloader:
            # Move batch to device
            inputs = batch['input_ids'].to(device)

            attention_mask = batch['attention_mask'].to(device)

            # Extract hidden states
            hidden_states = model.base_model.embeddings(inputs)

            # Apply Flash Attention
            attention_output = apply_flash_attention(hidden_states, model)

            torch.cuda.synchronize()
            start_time = time.time()
            # Perform final classification
            # Cast attention_output to float32 before passing it to the classifier
            outputs = model.classifier(attention_output.type(torch.float32))  # changed line
            logits = model.classifier(attention_output.mean(dim=1).type(torch.float32))
            probabilities = F.softmax(logits, dim=-1)

            predictions = torch.argmax(outputs, dim=1)
            all_predictions.extend(predictions.cpu().numpy())

            torch.cuda.synchronize()
            end_time = time.time()
            batch_inference_time = end_time - start_time
            inference_times.append(batch_inference_time)

            all_predictions.extend(probabilities.cpu().numpy())

    performance_metrics = {
        'total_samples': len(dataloader.dataset),
        'batch_size': dataloader.batch_size,
        'device': str(device),
        'inference_times': inference_times,
        'avg_batch_latency_ms': np.mean(inference_times) * 1000,
        'std_batch_latency_ms': np.std(inference_times) * 1000,
        'avg_sample_latency_ms': (np.mean(inference_times) * 1000) / dataloader.batch_size
    }

    return all_predictions, performance_metrics

def batch_inference_without_flash(dataloader, model, device):
    inference_times = []
    all_predictions = []

    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        for batch in dataloader:
            # Move batch to device
            inputs = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Extract hidden states
            hidden_states = model.base_model.embeddings(inputs)

            torch.cuda.synchronize()
            start_time = time.time()

            # Perform inference directly using the model
            outputs = model(inputs, attention_mask=attention_mask)

            # Get logits and apply softmax
            logits = outputs.logits
            probabilities = F.softmax(logits, dim=-1)
            predictions = torch.argmax(logits, dim=1)

            torch.cuda.synchronize()
            end_time = time.time()

            batch_inference_time = end_time - start_time
            inference_times.append(batch_inference_time)

            all_predictions.extend(predictions.cpu().numpy())

    performance_metrics = {
        'total_samples': len(dataloader.dataset),
        'batch_size': dataloader.batch_size,
        'device': str(device),
        'inference_times': inference_times,
        'avg_batch_latency_ms': np.mean(inference_times) * 1000,
        'std_batch_latency_ms': np.std(inference_times) * 1000,
        'avg_sample_latency_ms': (np.mean(inference_times) * 1000) / dataloader.batch_size
    }

    return all_predictions, performance_metrics


Let us run the experiment WITHOUT Flash Attention first
-------------------------------------------------------

In [10]:

# Configuration
batch_sizes = [32, 64, 128, 512, 1024, 2048]
device = 'cuda'  # Change to 'cpu' if necessary

# Load and preprocess the MRPC dataset
task = "mrpc"
dataset = load_dataset("glue", task)


for batch_size in batch_sizes:
    print(f"\n--- Batch Size: {batch_size} ---")

    # Create DataLoader
    test_dataset = MRPCDataset(dataset['test'], tokenizer)
    dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Perform batch inference on test set
    all_predictions, performance_metrics = batch_inference_without_flash(dataloader, model, device)

    # Print performance metrics
    print("\n--- Batch Inference Performance Metrics without Flash Attention---")
    for metric, value in performance_metrics.items():
        print(f"{metric}: {value}")



--- Batch Size: 32 ---

--- Batch Inference Performance Metrics without Flash Attention---
total_samples: 1725
batch_size: 32
device: cuda
inference_times: [0.4078843593597412, 0.037046194076538086, 0.037110328674316406, 0.037035226821899414, 0.031139612197875977, 0.031098604202270508, 0.031163692474365234, 0.031221389770507812, 0.031125307083129883, 0.03112053871154785, 0.031200170516967773, 0.0311279296875, 0.031178712844848633, 0.0311126708984375, 0.031129837036132812, 0.031122922897338867, 0.031180858612060547, 0.03164529800415039, 0.03121471405029297, 0.03121161460876465, 0.031185388565063477, 0.03120279312133789, 0.0311737060546875, 0.031145095825195312, 0.031026601791381836, 0.03120899200439453, 0.03123188018798828, 0.03124094009399414, 0.03114914894104004, 0.03112959861755371, 0.03113389015197754, 0.03119206428527832, 0.03119635581970215, 0.03146982192993164, 0.031152725219726562, 0.03115987777709961, 0.031213998794555664, 0.031200408935546875, 0.03116297721862793, 0.031157970

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 GiB. GPU 0 has a total capacity of 39.56 GiB of which 3.73 GiB is free. Process 30375 has 35.82 GiB memory in use. Of the allocated memory 31.73 GiB is allocated by PyTorch, and 3.60 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Now, let us run the Experiment WITH Flash Attention
----------------------------------------------------

In [12]:
for batch_size in batch_sizes:
    print(f"\n--- Batch Size: {batch_size} ---")

    # Create DataLoader
    test_dataset = MRPCDataset(dataset['test'], tokenizer)
    dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Perform batch inference on test set
    all_predictions, performance_metrics = batch_inference_with_flash(dataloader, model, device)

    # Print performance metrics
    print("\n--- Batch Inference Performance Metrics With Flash Attention ---")
    for metric, value in performance_metrics.items():
        print(f"{metric}: {value}")


--- Batch Size: 32 ---

--- Batch Inference Performance Metrics With Flash Attention ---
total_samples: 1725
batch_size: 32
device: cuda
inference_times: [0.0004608631134033203, 0.00039005279541015625, 0.0003669261932373047, 0.00035452842712402344, 0.0003533363342285156, 0.00036334991455078125, 0.0003516674041748047, 0.0003528594970703125, 0.0003478527069091797, 0.00035834312438964844, 0.00037217140197753906, 0.00034880638122558594, 0.0003876686096191406, 0.0003418922424316406, 0.00038695335388183594, 0.00036263465881347656, 0.0003619194030761719, 0.0003371238708496094, 0.00033354759216308594, 0.0003561973571777344, 0.0003647804260253906, 0.0003619194030761719, 0.0003437995910644531, 0.0003447532653808594, 0.00033092498779296875, 0.0003387928009033203, 0.00033926963806152344, 0.0003523826599121094, 0.0003619194030761719, 0.00036215782165527344, 0.00039005279541015625, 0.00037384033203125, 0.0003845691680908203, 0.0003447532653808594, 0.0003409385681152344, 0.0003409385681152344, 0.000